

 <h1>
Welcome to the Math Question Answer Verification Competition! 🚀

The goal is to fine-tune a Llama-3-8B model to predict if a given solution to a math problem is correct or not. Your model should output True if the solution is correct, and False otherwise.

This notebook is a starter guide designed to get you up and running quickly. We'll walk through a simplified training process using a small subset of the data (5,000 examples) and lightweight parameters. The main goal here is to understand the complete workflow, from loading data to generating a submission file, not to achieve a top score.

Good luck, and have fun! 🎉

## **Step 1: Install Necessary Libraries**

First, we need to install the required Python libraries. We'll be using the unsloth library, which provides highly efficient, memory-saving training methods for large language models, making it possible to fine-tune powerful models on a single free-tier GPU. We'll also install xformers for further optimization.


## **Step 2: Load the Model and Tokenizer**

Next, we'll load the Llama-3-8B model, which is the only model permitted for this competition. We'll use Unsloth's FastLanguageModel to handle this efficiently.

A key technique we'll use is 4-bit quantization (load_in_4bit = True). Think of this as compressing the model's knowledge into a much smaller file size. This significantly reduces the amount of GPU memory required, allowing us to fine-tune this large model even on a free platform like Google Colab.



In [4]:
import unsloth

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


[torchao|WARNING]Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info


🦥 Unsloth Zoo will now patch everything to make training faster!


In [10]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Note: We use the base model, not a 4-bit pre-quantized one,
# to ensure we start from the official weights.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # Competition-approved model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 4070 Ti. Num GPUs = 1. Max memory: 11.593 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

## **Step 3: Prepare the Dataset**

This is a crucial step where we format our data into a structure the model can learn from. The process involves three parts:

1.  **Loading**: We'll load the official competition dataset from Hugging Face.
2.  **Splitting**: The full dataset is massive. For this starter notebook, we'll create a much smaller, more manageable version to speed things up: **5,000 samples for training** and **500 for validation**.
3.  **Prompting**: We will format each data sample into a clear instructional prompt. This helps the model understand its role as a mathematician verifying a solution.



In [9]:
from datasets import load_dataset, DatasetDict, ClassLabel
import re, unicodedata, hashlib, math
from typing import Dict

# Step 1: Clean code and markdown in text
def strip_code_fences(text):
    if not isinstance(text, str): return ""

    # delete markdown block
    text = re.sub(r"```.*?```", "", text, flags=re.DOTALL)

    # delete code block
    text = re.sub(r"`+", "", text)
    return text.strip()

# Step 2: Normalize the numbers and related representation
def normalize_numbers(text):
    if not isinstance(text, str):
        return ""
    t = unicodedata.normalize("NFKC", text)
    t = t.replace("≈", "=")                   # clean ≈
    t = re.sub(r"\s+", " ", t).strip()        # clean blank
    # decimal palces to 4
    t = re.sub(r"(\d+\.\d{4})\d+", r"\1", t)
    # delete the comma 1,234.56 -> 1234.56
    t = re.sub(r"(?<=\d),(?=\d{3}\b)", "", t)
    return t

# Step 3: Delete HTML and LaTex sign
def strip_markup(text):
    if not isinstance(text, str):
        return ""
    # delete HTML label <...>
    text = re.sub(r"<[^>]+>", " ", text)
    # delete LaTeX sign $...$
    text = re.sub(r"\$(.*?)\$", r"\1", text)
    return text

# Step 5: Normalize as bool；(Some source are 'True'/'False'/1/0/true/false)
def boolify_label(x):
    if isinstance(x, bool):
        return x
    if isinstance(x, (int, float)):
        return bool(x)
    if isinstance(x, str):
        return x.strip().lower() in {"true", "1", "yes", "y", "t"}
    return False

def normalize_text(s):
    s = s or ""
    s = strip_code_fences(s)
    s = strip_markup(s)
    s = normalize_numbers(s)
    return s

In [7]:
def preprocess_row(ex):
    q = normalize_text(ex.get("question", ""))
    a = normalize_text(ex.get("answer", ""))
    o = normalize_text(ex.get("is_correct", ""))
    s = normalize_text(ex.get("solution", ""))

In [7]:
from datasets import load_dataset

# Load the full training dataset
train_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# Shuffle the dataset for randomness and create our smaller splits
# shuffled_dataset = full_dataset.shuffle(seed=42)
# train_dataset = shuffled_dataset.select(range(5000))      # Use the first 5,000 for training
# validation_dataset = shuffled_dataset.select(range(5000, 5500)) # Use the next 500 for validation

In [8]:
# The instructional prompt template for training
training_prompt = (
    "You are a great mathematician and you are tasked with determining whether "
    "the given answer to a math question is correct. "
    "Respond with True if the answer is correct, otherwise False.\n\n"
    "Question:\n{question}\n"
    "Given Answer:\n{answer}\n"
    "Solution:\n{solution}\n"
    "Output:\n{output}"
)

EOS_TOKEN = tokenizer.eos_token or ""

# Force converting to True/False
def to_bool_str(x):
    s = str(x).strip().lower()
    return "True" if s in {"true", "1", "yes", "y", "t"} else "False"


def formatting_prompts_func(examples):

    Q = examples.get("question_norm", examples.get("question", []))
    A = examples.get("answer_norm",   examples.get("answer",   []))
    S = examples.get("solution_norm", examples.get("solution", []))
    Y = examples.get("is_correct",    [])

    texts = []
    n = min(len(Q), len(A), len(S), len(Y))
    for i in range(n):
        text = training_prompt.format(
            question=str(Q[i]),
            answer=str(A[i]),
            solution=str(S[i]),
            output=to_bool_str(Y[i])
        ) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# clean training data
train_dataset_clean = train_dataset.map(
    preprocess_row,
    desc="Clean training data"
)


formatted_train_dataset = train_dataset_clean.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=[c for c in train_dataset_clean.column_names if c != "text"],
    desc="Format cleaned training prompts"
)

print("\nExample cleaned train sample:\n")
print(formatted_train_dataset[0]["text"])

NameError: name 'tokenizer' is not defined

In [ ]:
# ################Skip##################################
# # The instructional prompt template for training
# training_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
# Question:
# {}
# Solution:
# {}
# Output:
# {}"""

# # We must add an End Of Sequence (EOS) token to tell the model when a completion is finished.
# EOS_TOKEN = tokenizer.eos_token

# # This function formats our data samples into the prompt template.
# def formatting_prompts_func(examples):
#     questions = examples["question"]
#     solutions = examples["solution"]
#     outputs = examples["is_correct"]
#     texts = []
#     for question, solution, output in zip(questions, solutions, outputs):
#         # Format the prompt and add the EOS token
#         text = training_prompt.format(question, str(solution), str(output)) + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts }

# # Apply the formatting function to our training dataset
# formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)

In [ ]:
# validation_dataset_clean = validation_dataset.map(
#     preprocess_row,
#     desc="Clean validation data",
# )

## **Step 4: Configure LoRA and Set Up the Trainer**

### **LoRA Configuration**

Instead of training the entire model (which has billions of parameters), we'll use a technique called **Lo**w-**R**ank **A**daptation (LoRA). 🎛️

Think of it like this: rather than rewriting an entire textbook, we're just adding small, efficient "sticky notes" (the LoRA adapters) to update the model's knowledge. This is much faster and requires significantly less memory. We'll use a small **rank** (`r = 8`) to keep the training process light and quick for this starter notebook.


In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # A small rank for lighter training
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16, # A common practice is to set alpha = 2 * r
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.10.11 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.



### **SFTTrainer Setup**

Now we'll set up the `SFTTrainer` (Supervised Fine-tuning Trainer). This is the main tool from the `trl` library that will handle the entire training loop for us. We'll give it our model, tokenizer, dataset, and a set of training instructions, such as the batch size and number of epochs.

We will train for just **one epoch** (a single pass over our 5,000-sample dataset) to keep this demonstration fast.

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        max_steps = 15000,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 100,
        seed = 42,
        output_dir = "outputs",
        report_to = "none",

        # 🏆 Best Hyperparameters
        optim = "adamw_torch",
        learning_rate = 5e-4,
        lr_scheduler_type = "cosine_with_restarts",
        warmup_ratio = 0.1,
        weight_decay = 0.03,
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=24): 100%|██████████| 1000000/1000000 [00:35<00:00, 27991.47 examples/s]


## **Step 5: Start Training\!**

Now, we'll call the `train()` function on our `trainer` object. This will kick off the fine-tuning process. Based on our settings, this will run for one full epoch over our 5,000 examples.

Grab a coffee, as this will take a few minutes\! ☕


In [11]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000,000 | Num Epochs = 1 | Total steps = 15,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
100,1.185700
200,0.763100
300,0.742700
400,0.718600
500,0.709300
600,0.724000
700,0.706900
800,0.719200
900,0.709900
1000,0.697900


TrainOutput(global_step=15000, training_loss=0.572453049214681, metrics={'train_runtime': 39777.4466, 'train_samples_per_second': 3.017, 'train_steps_per_second': 0.377, 'total_flos': 1.9010753871965553e+18, 'train_loss': 0.572453049214681, 'epoch': 0.12})

In [ ]:
# from itertools import product
# from tqdm import tqdm
# import torch
# import numpy as np
# import pandas as pd
# from trl import SFTTrainer
# from transformers import TrainingArguments
# from unsloth import FastLanguageModel

# # -----------------------------
# # 📘 基础设置
# # -----------------------------
# inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not.
# Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
# Question:
# {}
# Solution:
# {}
# Output:
# """

# # -----------------------------
# # 🔧 网格搜索空间
# # -----------------------------
# learning_rates = [1e-4, 2e-4, 5e-4]
# optimizers = ["adamw_torch", "sgd", "adamw_torch_fused"]
# warmup_ratios = [0.05, 0.1, 0.2]

# # -----------------------------
# # 📊 存储结果
# # -----------------------------
# results = []

# # -----------------------------
# # 🚀 循环搜索每组超参数
# # -----------------------------
# for lr, opt, wr in product(learning_rates, optimizers, warmup_ratios):
#     print(f"\n🚀 Training with lr={lr}, optimizer={opt}, warmup_ratio={wr}")
#     print("-" * 60)

#     # ⚙️ 重新创建轻量 LoRA 模型，避免梯度污染
#     temp_model = FastLanguageModel.get_peft_model(
#       model,
#       r = 16, # A small rank for lighter training
#       target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                         "gate_proj", "up_proj", "down_proj"],
#       lora_alpha = 16, # A common practice is to set alpha = 2 * r
#       lora_dropout = 0,
#       bias = "none",
#       use_gradient_checkpointing = "unsloth",
#       random_state = 3407,
#       use_rslora = False,  # We support rank stabilized LoRA
#       loftq_config = None, # And LoftQ
#     )

#     # 🧠 训练器定义
#     trainer = SFTTrainer(
#         model=temp_model,
#         tokenizer=tokenizer,
#         train_dataset=formatted_train_dataset,
#         dataset_text_field="text",
#         max_seq_length=max_seq_length,
#         args=TrainingArguments(
#             per_device_train_batch_size=2,
#             gradient_accumulation_steps=8,
#             warmup_ratio=wr,
#             max_steps=50,
#             learning_rate=lr,
#             fp16=not torch.cuda.is_bf16_supported(),
#             bf16=torch.cuda.is_bf16_supported(),
#             logging_steps=10,
#             optim=opt,
#             weight_decay=0.01,
#             lr_scheduler_type="linear",
#             seed=3407,
#             output_dir=f"outputs/lr{lr}_opt{opt}_wr{wr}",
#             report_to="none",
#         ),
#     )

#     trainer.train()

#     # -----------------------------
#     # 🧪 推理阶段
#     # -----------------------------
#     FastLanguageModel.for_inference(temp_model)
#     temp_model.eval()

#     correct, total = 0, 0

#     for example in tqdm(validation_dataset_clean.select(range(200))):
#         question = example["question"]
#         solution = example["solution"]
#         true_label = example["is_correct"]

#         prompt = inference_prompt.format(question, str(solution))
#         inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

#         with torch.no_grad():
#             outputs = temp_model.generate(**inputs, max_new_tokens=8, use_cache=True)

#         response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
#         response_text = response.split("Output:")[-1].strip().lower()

#         predicted_label = None
#         if "true" in response_text:
#             predicted_label = True
#         elif "false" in response_text:
#             predicted_label = False

#         if predicted_label is not None:
#             total += 1
#             if predicted_label == bool(true_label):
#                 correct += 1

#     accuracy = correct / total if total > 0 else 0
#     loss = trainer.state.log_history[-1].get("loss", None)

#     print(f"✅ Accuracy: {accuracy*100:.2f}% ")

#     # 保存结果
#     results.append({
#         "learning_rate": lr,
#         "optimizer": opt,
#         "warmup_ratio": wr,
#         "accuracy": accuracy,

#     })

# # -----------------------------
# # 📈 结果汇总
# # -----------------------------
# df = pd.DataFrame(results)
# print("\n📊 Grid Search Results:")
# print(df.sort_values(by="accuracy", ascending=False))

# best = df.loc[df["accuracy"].idxmax()]
# print(f"\n🏆 Best Hyperparameters:\n{best}")



## **Step 6: Inference and Evaluation**

Now that our model is trained, we need to test it on our validation set. We'll use a slightly different prompt for inference—one where we leave the `Output:` section blank for the model to complete.

Let's test it on a single example from our validation set to see what it predicts.

In [ ]:
# Prepare the model for faster inference
FastLanguageModel.for_inference(model)

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
"""

# Select a sample from the validation set
example = validation_dataset[10] # You can change the index (e.g., to 1, 2, 50)
question = example["question"]
solution = example["solution"]

# Format the prompt with the validation data
inputs = tokenizer(
[
    inference_prompt.format(question, str(solution))
], return_tensors = "pt").to("cuda")

# Generate the model's response
outputs = model.generate(**inputs, max_new_tokens = 8, use_cache = True)
response = tokenizer.batch_decode(outputs)

# Print the results
print("#### QUESTION ####")
print(question)
print("\n#### SOLUTION ####")
print(solution)
print("\n#### MODEL'S PREDICTION ####")
# We process the output to show only the generated text
print(response[0].split("Output:\n")[1])
print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])

In [ ]:
FastLanguageModel.for_inference(model)
model.eval()
inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
"""
# 统计正确样本数
correct = 0
total = 0

for example in tqdm(validation_dataset):  # 这里换成你的测试集
    question = example["question"]
    solution = example["solution"]
    true_label = example["is_correct"]  # True/False 或 1/0

    # 构建 prompt
    prompt = inference_prompt.format(question, str(solution))

    # 编码输入
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # 生成模型输出
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)

    # 解码模型输出
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 提取 "Output:" 后面的部分
    if "Output:" in response:
        response_text = response.split("Output:")[-1].strip()
    else:
        response_text = response.strip()

    # 标准化模型输出
    response_text = response_text.lower().replace(".", "").strip()
    predicted_label = None
    if "true" in response_text:
        predicted_label = True
    elif "false" in response_text:
        predicted_label = False

    # 统计准确率
    if predicted_label is not None:
        total += 1
        if predicted_label == bool(true_label):
            correct += 1

accuracy = correct / total if total > 0 else 0
print(f"✅ Test Accuracy: {accuracy*100:.2f}% ({correct}/{total})")

## **Step 7: Generate Submission File**

This is the final step\! We will now run our fine-tuned model on the official `test` dataset.

We will loop through each example in the test set, generate a prediction, and format the results into a CSV file with two columns: `ID` and `is_correct`, as required by the competition.


In [ ]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# Create the prompt template for inference (no answer included)
inference_prompt = (
    "You are a great mathematician and you are tasked with determining whether "
    "the given answer to a math question is correct. "
    "Respond with True if the answer is correct, otherwise False.\n\n"
    "Question:\n{}\n"
    "Given Answer:\n{}\n"
    "Solution:\n{}\n"
    "Output:\n"
)

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission5.csv', index=False)

print("\nSubmission file 'submission5.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 4070 Ti. Num GPUs = 1. Max memory: 11.593 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model and tokenizer loaded from: llama3_8b_math_verifier_checkpoint20251101120030


100%|██████████| 10000/10000 [27:21<00:00,  6.09it/s]


Submission file 'submission5.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.


# SAVE THE MODEL TO DRIVE AND RUN INFERENCE
Add code to save the model checkpoint to Google Drive, load the model from the checkpoint, and generate the final submission CSV file.

## Save model checkpoint

### Subtask:
Save the trained model checkpoint to the specified path in Google Drive.


**Reasoning**:
Define the save path and save the model and tokenizer to Google Drive.



In [13]:
import os
from datetime import datetime

# Define the path to save the model checkpoint in Google Drive
save_path = "llama3_8b_math_verifier_checkpoint"+datetime.now().strftime("%Y%m%d%H%M%S")


# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

Model checkpoint and tokenizer saved to: llama3_8b_math_verifier_checkpoint20251101120030


## Load model from checkpoint

### Subtask:
Load the model from the saved checkpoint.


**Reasoning**:
Load the model and tokenizer from the saved checkpoint path in Google Drive and prepare the model for inference.



In [ ]:
# # Define the path where the model checkpoint was saved in Google Drive
# save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"

# # Load the model and tokenizer from the saved path
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = save_path,
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )

# # Prepare the loaded model for faster inference
# FastLanguageModel.for_inference(model)

# print(f"Model and tokenizer loaded from: {save_path}")

## Generate submission file

### Subtask:
Generate the submission CSV file using the loaded model.


**Reasoning**:
Generate the submission CSV file by iterating through the test dataset, generating predictions using the loaded model, and saving the results to a pandas DataFrame.



In [1]:
from unsloth import FastLanguageModel


# save_path = "llama3_8b_math_verifier_checkpoint_20251031055355"
save_path = "llama3_8b_math_verifier_checkpoint20251101120030"

max_seq_length = 2048  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Note: We use the base model, not a 4-bit pre-quantized one,
# to ensure we start from the official weights.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # Competition-approved model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Prepare the loaded model for faster inference
FastLanguageModel.for_inference(model)

print(f"Model and tokenizer loaded from: {save_path}")

import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
"""

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission5.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/victoria-gao/Projects/DL/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[torchao|WARNING]Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 4070 Ti. Num GPUs = 1. Max memory: 11.593 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model and tokenizer loaded from: llama3_8b_math_verifier_checkpoint20251101120030


100%|██████████| 10000/10000 [30:17<00:00,  5.50it/s]


Submission file 'submission.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.
